<a href="https://colab.research.google.com/github/michalwilk123/huggingface-bert-finetune-example-pl/blob/master/ProjektSI_modele_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tworzenie nowego modelu BERT za pomocą techniki fine-tune

In [ ]:
! pip install transformers datasets torch &> /dev/null
from datasets import load_dataset, Dataset, DatasetDict

# Zbiór danych

In [ ]:
dataset = load_dataset("imdb")

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b. Subsequent calls will reuse this data.


## Rozdzielanie danych na dwa spolaryzowane podzbiory

In [ ]:
def split_polarity_based(dataset) -> tuple:
    """
    Splitting dataset basing on Label. The expected output of this function
    is tuple of two datasets: negative and positive (0 = neg, 1 = pos)
    The inner structure of datasets is preserved
    """
    negative, positive = DatasetDict(), DatasetDict()

    for k in dataset:
        if k == 'unsupervised':
            continue
        negative[k] = dataset[k].filter(lambda dat: dat['label'] == 0)
        positive[k] = dataset[k].filter(lambda dat: dat['label'] == 1)
    return negative, positive

neg_dataset, pos_dataset = split_polarity_based(dataset)

Deklarujemy nasz tokenizer oraz model: DistilBERT, który jest 'okrojoną' wersją berta, która dla naszych celów powinna w zupełności wystarczyć.

Oprócz tego musimy w jakiś sposób przetrenować nasz model i wypełnić testowe dane 'dziurami', które nasz model powinien wypełnić. Wy tym celu deklarujemy
narzędzie o nazwie __DataCollatorForLanguageModeling__, które z  prawdopodobieństwem o wartości __mlm_probability__ zamaskuje nam wyrazy w zdaniach.

Podana tutaj wartość prawdopodobieństwa jest zgodna z tą sugerowaną w [pracy naukowej](https://arxiv.org/abs/1810.04805): 

In [ ]:
from transformers import (AutoTokenizer, 
                          AutoModelForMaskedLM, 
                          DataCollatorForLanguageModeling)
import torch

MODEL_NAME = "distilbert-base-uncased"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, use_fast=True
)

model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME).to(device)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm_probability=0.15
)

Teraz musimy zmienic tekst typu: "Ala ma kota", na jego reprezentacje
za pomocą tokenów, czyli np: 1, 10, 8.

Jako że zadaniem naszych modeli __nie__ jest klasyfikacja (Sentiment analysis), tylko wypełnianie pustych miejsc w zdaniach, to pola 'label' określające nastrój (polaryzacje) nie są już nam więcej potrzebne

In [ ]:
tokenized_negative = neg_dataset.map(
    lambda dat:tokenizer(dat['text'], truncation=True), 
    batched=True, num_proc=4, remove_columns=["text", "label"]
)
tokenized_positive = pos_dataset.map(
    lambda dat:tokenizer(dat['text'], truncation=True), 
    batched=True, num_proc=4, remove_columns=["text", "label"]
)

# Trenowanie modeli


Kolejnym krokiem jest faktyczne trenowanie modelu na naszych danych!

Korzystamy w tym celu z narzędzia o nazwie __Trainer__

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "modele-negatywne",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_negative['train'],
    eval_dataset=tokenized_negative["test"],
    data_collator=data_collator,
)

In [ ]:
# Odkomentuj poniższą linijkę aby wykonać 'fine-tune' na podanych danych
# Estymowany czas treningu i ewaluacji modelu: ~2h
# trainer.train()

# Odkomentuj poniższe linijki aby zapisać przetrenowany model na 
# lokalnym folderze
# model.save_pretrained("distilbert-imdb-negative")
# tokenizer.save_pretrained("distilbert-imdb-negative")

In [ ]:
training_args = TrainingArguments(
    "modele-pozytywne",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_positive['train'],
    eval_dataset=tokenized_positive["test"],
    data_collator=data_collator,
)

In [ ]:
# Odkomentuj poniższą linijkę aby wykonać 'fine-tune' na podanych danych
# Estymowany czas treningu i ewaluacji modelu: ~2h
# trainer.train()

# Odkomentuj poniższe linijki aby zapisać przetrenowany model na 
# lokalnym folderze
# model.save_pretrained("distilbert-imdb-positive")
# tokenizer.save_pretrained("distilbert-imdb-positive")

Epoch,Training Loss,Validation Loss
1,2.339600,2.216184
2,2.267100,2.174873
3,2.233800,2.157197


('distilbert-imdb-positive/tokenizer_config.json',
 'distilbert-imdb-positive/special_tokens_map.json',
 'distilbert-imdb-positive/vocab.txt',
 'distilbert-imdb-positive/added_tokens.json',
 'distilbert-imdb-positive/tokenizer.json')

# Pobieranie stworzonego negatywnego modelu z platformy huggingface

*   [Udostępniony model pozytywny](https://huggingface.co/michalwilk123/distilbert-imdb-positive)
*   [Udostępniony model negatywny](https://huggingface.co/michalwilk123/distilbert-imdb-negative)



Aby za każdym razem nie musieć pobierać ani martwić się o lokalizacje modelu, 
udostępniłem go na platformie huggingface.

Poniżej pobieram udostępniony model, przez co powyższe trenowanie nie jest już nam więcej potrzebne

In [ ]:
! pip install transformers datasets torch &> /dev/null
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# określamy na jakim urządzeniu będzie oodtwarzany model:
# * cpu: na procesorze
# * cuda: na kartach graficznych z architekturą nVidia
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

tokenizer = AutoTokenizer.from_pretrained("michalwilk123/distilbert-imdb-negative", use_fast=True)

model = AutoModelForMaskedLM.from_pretrained("michalwilk123/distilbert-imdb-negative").to(device)

# Testowanie negatywnego modelu

In [ ]:
text = "This movie is " + tokenizer.mask_token + "!" # nasze zdanie które chcemy testować
inputs = tokenizer(text, return_tensors = "pt").to(device) # tworzymy listę tokenów ze zdania
# zapamiętujemy lokacje zamaskowanego wyrazu (tokenizer może dodawać dodatkowe specjalne tokeny)
mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)
outputs = model(**inputs) # przerzucamy nasze zdanie przez model

In [ ]:
mask_word = outputs.logits[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]

# wyświetlamy 10 najbardziej prawdopodobnych wyrazów
for token in top_10:
   word = tokenizer.decode(token)
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

This movie is terrible!
This movie is bad!
This movie is awful!
This movie is crap!
This movie is unbelievable!
This movie is horrible!
This movie is ridiculous!
This movie is great!
This movie is stupid!
This movie is rubbish!


# Pobieranie stworzonego pozytywnego modelu z platformy huggingface

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  
tokenizer = AutoTokenizer.from_pretrained("michalwilk123/distilbert-imdb-positive", use_fast=True)

model = AutoModelForMaskedLM.from_pretrained("michalwilk123/distilbert-imdb-positive").to(device)

# Testowanie pozytywnego modelu

Korzystamy z modelu w następujący sposób:

Podajemy nasz stokenizowany tekst do funkcji forward przetrenowanego modelu.
Jako rezultat tej funkcji otrzymujemy tensor(czyli jakby listę) zawierającej
wagi prawdopodobieństw wszystkich słów z naszego słownika.

Tak więc podając na samym początku słownik z N słowami otrzymamy tensor o wymiarach N x 1.
Ich wartość odzwierciedla __prawdopodobieństwo__ wyboru danego słowa w danym 
miejscu. Jednak nie jest to nasza tradycyjna definicja prawdopodobieństwa,
ponieważ może przyjmować wartości spoza przedziału [0, 1].

In [ ]:
text = "This movie is " + tokenizer.mask_token + "!"
inputs = tokenizer(text, return_tensors = "pt").to(device)
mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)
outputs = model(**inputs)
mask_word = outputs.logits[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]

for token in top_10:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

This movie is great!
This movie is amazing!
This movie is incredible!
This movie is awesome!
This movie is excellent!
This movie is wonderful!
This movie is brilliant!
This movie is hilarious!
This movie is good!
This movie is fantastic!


# Pobieranie i testowanie modelu klasyfikatora

Tutaj warto by przerobić nasz rezultat na naszą tradycyjną definicje prawdopodobieństwa.
Model zwraca tensor wag generowanych przez funkcje logitową. Aby zmienić wartość funkcji logitowej na nasze tradycyjne prawdopodobieństwo od 0 do 1 wrzucamy nasze wartości do funkcji __softmax__.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb").to(device)

In [ ]:
from torch.nn import functional as F

negative_sen = "This movie is awful!"
positive_sen = "This movie is awesome!"

outputs_n = model(**tokenizer(negative_sen, return_tensors = "pt").to(device))
nch = F.softmax(outputs_n['logits'], dim=-1).tolist()

outputs_p = model(**tokenizer(positive_sen, return_tensors = "pt").to(device))
pch = F.softmax(outputs_p['logits'], dim=-1).tolist()

print(
    "NO |   N  | P\n",
    "1 | %.2f | %.2f\n" % (round(nch[0][0], 2), round(nch[0][1], 2)),
    "2 | %.2f | %.2f\n" % (round(pch[0][0], 2), round(pch[0][1], 2)),
)

NO |   N  | P
 1 | 1.00 | 0.00
 2 | 0.05 | 0.95

